# Quote Generation/Retrieval

Initial quotes were scraped from one webpage. But that would lead to quite a lot of repitition and kindof a boring bot. So...

### 2 Issues with Index retrieved/Generated Quotes:
**Goal:** To have ~100 quotes to have tweeted on a rolling basis since this is a quarter of a year and will not likely to be easily repeated.

### Probability of Repeating a Quote Exactly 2 Weeks Later

\begin{equation*}
P(\text{repeated in 2 weeks}) = 1 - \left( \frac{N - 1}{N} \right)^{\frac{14}{m}}
\end{equation*}

 Substituting in the values:

 \begin{equation*}
P(\text{repeated in 2 weeks}) = 1 - \left( \frac{99}{100} \right)^{14}
\end{equation*}

 Calculating this gives:

 \begin{equation*}
P(\text{repeated in 2 weeks}) \approx 0.135
\end{equation*}

Personally, I'd like it to be under 10% for a longer period of time (increases probability)

Tweaking the numbers a bit 
N = 200
days = 21

\begin{equation*}
P(\text{repeated in 3 weeks}) \approx  0.0980
\end{equation*}

The following should be conditional 1) That the quote generated has confidence that the quote was written by Ovid then 2) That the quote is not a variation of a quote already in DB.

In [4]:
from langchain.document_loaders import PyPDFLoader
from langchain.vectorstores import FAISS
from langchain.chat_models import ChatOpenAI
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.chains import RetrievalQA, ConversationalRetrievalChain
import os
import openai
import pandas as pd

from itertools import product

# import utils
import configparser
from pprint import pprint
from llama_index import SimpleDirectoryReader


import os
import openai


import jupyter_black

jupyter_black.load()
# Access values from the sections

import sys

sys.path.append("../app")
import conn_utils

OPENAI_API_KEY = conn_utils.get_open_ai_key("./../app/config.ini")

os.environ["OPENAI_API_KEY"] = OPENAI_API_KEY
directory = "index_store"

pd.set_option("display.max_colwidth", None)

In [13]:
!ls ./../data

AmoresV2.txt           LostPoemsV2.txt        RemediaAmorisV2.txt
ArsAmatoria.txt        LoversAssistant.txt    notes.jsonl
FastiV2.txt            MetamorphosesV2.txt    notes_validation.jsonl
HeroidesV2.txt         PoemsFromExile.txt


## Get low sentency similarity

Instead of using a LLM. You can trust to get a close (but not too close) similarity

In [5]:
import spacy

# Read in exisitng quotes
df = pd.read_json("./../app/ovid_quotes.json")

# Load the spaCy English model
nlp = spacy.load("en_core_web_lg")

# Define the file paths
f1 = "./../data/FastiV2.txt"
f2 = "./../data/MetamorphosesV2.txt"
f3 = "./../data/AmoresV2.txt"
f4 = "./../data/LostPoemsV2.txt"
f5 = "./../data/HeroidesV2.txt"
f6 = "./../data/ArsAmatoria.txt"
f7 = "./../data/RemediaAmorisV2.txt"
f8 = "./../data/LoversAssistant.txt"

# Read in the text files and split sentences
from tqdm import tqdm

# Override the max length of the text
nlp.max_length = 1500000  # or any value that accommodates your text length

with open(os.path.join("./../data", "PoemsFromExile.txt"), "r") as f:
    text = f.read()
doc = nlp(text)
sentences = [sent.text for sent in doc.sents]
# Print the list of sentences
print(f"Num sentences: {len(sentences)}")
pprint(sentences[:5])

Num sentences: 2920
['Ovid: The Poems Of Exile\n'
 '(Tristia, Ex Ponto, Ibis)\n'
 '\n'
 'Home\n'
 '\n'
 'Download\n'
 '\n'
 '\x0c'
 'Translated by A. S. Kline 2003\n'
 'All Rights Reserved\n'
 'This work may be freely\n'
 'reproduced, stored, and\n'
 'transmitted, electronically or\n'
 'otherwise, for any non-commercial\n'
 'purpose.\n'
 '\n',
 '2\n'
 '\n'
 '\x0c'
 'Contents\n'
 'Tristia Book '
 'I.................................................................. 11\n'
 'Book TI.I:1-68 The Poet to His Book: Its Nature ...........',
 '11\n'
 'Book TI.I:70-128 The Poet to His Book: His Works...... 14\n'
 'Book TI.II:1-74 The Journey: Storm at Sea.................... 17\n'
 'Book TI.II:75-110 The Journey: The Destination........... 21\n'
 'Book TI.III:1-46 The Final Night in Rome:',
 'Preparation 23\n'
 'Book TI.III:47-102 The Final Night in Rome: Departure25\n'
 'Book TI.IV:1-28 Troubled Waters.................................. 28\n'
 'Book TI.V:1-44 Loyalty in Friendship ...............

In [21]:
# import pandas as pd

# df = pd.read_json("./../app/ovid_quotes.json")
# ## Update add new quotes to the dataframe
# new_row = {
#     "Quote": """Forgive me, please, excuse excessive dread.""",
#     "Work": "Tristia",
#     "Quote in Latin": "",
# }
# df = pd.concat([df, pd.DataFrame([new_row])], ignore_index=True)
# df.to_json("./../app/ovid_quotes.json")
# df.shape

(57, 3)

In [49]:
df.sample(5)

,Quote,Quote in Latin,Work
13,They bear punishment with equanimity who have earned it.,"Aequo animo poenam, qui meruere, ferunt.","Amores (Love Affairs), Book II, vii, 12"
37,"Chance is always powerful. Let your hook be always cast; in the pool where you least expect it, there will be a fish.","Semper tibi pendeat hamus: quo minime credas gurgite, piscis erit.",Heroides
23,"Everything changes, nothing perishes.",None,Metamorphoses
35,Let us that love both hope and fear in equal degree and let an occasional repulse make room for our desires.,"Amemus, et pares spes metusque sint, et raro cedat in vota libido.",Amours
26,It is no use to blame the looking glass if your face is awry.,None,Amores


In [6]:
import re


def clean_sentences(sentences, filter_tokens):
    def replace_non_alphanumeric(input_string):
        processed_text = re.sub(r"\n", " ", input_string)
        processed_text = re.sub(r"[^a-zA-Z0-9\s.,?!'\-]", " ", processed_text)
        return processed_text

    def contains_filter_tokens(input_string):
        return any(token in input_string for token in filter_tokens)

    cleaned_sentences = [s.replace("\n", ". ").replace("_", " ") for s in sentences]
    cleaned_sentences = [replace_non_alphanumeric(s) for s in sentences]
    cleaned_sentences = [s for s in cleaned_sentences if len(s.split()) > 4]
    cleaned_sentences = [s for s in cleaned_sentences if not contains_filter_tokens(s)]

    return cleaned_sentences


# Example usage:
print(len(sentences))
filter_tokens = [
    "Project Gutenberg",
    "Footnote",
    "--Ver.",
    "The Amores",
    "United States",
    "copyright",
    "Copyright",
    "gutenberg",
    "Gutenberg",
]
cleaned = clean_sentences(sentences, filter_tokens)
print(len(cleaned))

# Uncomment to use the clean sentences
sentences = clean_sentences(sentences, filter_tokens)

2920
2864


In [8]:
import random

sampled_cell = random.choice(cleaned)
print(sampled_cell)

The gods delight in instances of such testimony, since they, thereby, give witness of their powers. 


In [9]:
corpus_text = text
# The string you want to find
target_string = "barbarian"

# Find the index of the target string
start_index = corpus_text.find(target_string)

# Ensure the target string is found in the corpus
if start_index != -1:
    # Extract the text before and after the target string
    text_before = corpus_text[:start_index].strip()
    text_after = corpus_text[start_index + len(target_string) :].strip()

    # Tokenize the sentences using spaCy
    doc_before = nlp(text_before)
    doc_after = nlp(text_after)

    # Get the sentences
    sentences_before = [sent.text for sent in doc_before.sents][-2:]
    sentences_after = [sent.text for sent in doc_after.sents][:2]

    # Print or use the extracted sentences
    print("Sentences Before:")
    print(sentences_before)

    print("\nSentences After:")
    print(sentences_after)
else:
    print("Target string not found in the corpus.")

Sentences Before:
['So, a suppliant, I beg you to banish me somewhere safe,\nso that peace as well as my home aren’t taken from me,\nso as not to fear the tribes the Danube scarcely checks,\nso your subject can’t be captured by the enemy.\n', 'Justice forbids any man of Roman blood\nto suffer']

Sentences After:
['chains while Caesars live.\n\n58\n\n\x0cBook TII:207-252 His Plea: ‘Carmen et Error’\n', 'Though two charges, carmen et error, a poem and an\nerror,\nruined me, I must be silent about the second fault:\nI’m not important enough to re-open your wound, Caesar,\nit’s more than sufficient you should be troubled once.\n']


In [22]:
similarity_data = []

# Sample similarity quotes
# But I won t refute a thing  I favour your praise too  For, heart, why reject the voice that is desired? Don t be angry if my belief in you comes only with great  difficulty  trust in important things usually builds slowly.


quotes = [
    """The tongue is the instrument of the greatest good and the greatest evil that is in man."""
]
for actual_quote in quotes:
    doc_actual = nlp(actual_quote)

    for sent in tqdm(sentences):
        doc_generated = nlp(sent)
        # Calculate similarity score
        similarity_score = doc_generated.similarity(doc_actual)

        # Append data to the list
        similarity_data.append(
            {
                "new_quotes": doc_generated.text,
                "actual_quotes": actual_quote,
                "similarity_score": similarity_score,
            }
        )
# Create a DataFrame from the similarity data
similarity_df = pd.DataFrame(similarity_data).sort_values(
    "similarity_score", ascending=False
)
display(similarity_df.head(1))

  0%|          | 0/2864 [00:00<?, ?it/s]

100%|██████████| 2864/2864 [00:25<00:00, 112.18it/s]


,new_quotes,actual_quotes,similarity_score
949,"The Father of the Country himself and who is milder than him tolerates being mentioned often in my verse, nor can he prevent it, since Caesar is the State, and a share of the common wealth is also mine.",The tongue is the instrument of the greatest good and the greatest evil that is in man.,0.936098


In [23]:
similarity_df.sample(20)

,new_quotes,actual_quotes,similarity_score
47,206 Book EI.VI 1-54 To Graecinus Hope ..........................,The tongue is the instrument of the greatest good and the greatest evil that is in man.,-0.039151
2147,"I wrote for those whose chaste hair was never touched by ribbons, nor their feet by the long robe.",The tongue is the instrument of the greatest good and the greatest evil that is in man.,0.707655
1517,"Why tell of the ancient Romans, whose furthest place of exile was only Tibur?",The tongue is the instrument of the greatest good and the greatest evil that is in man.,0.862955
2796,"May you approach high places no more safely than Elpenor, and suffer the effects of wine in the same way he did. And die as tamely, as whoever delighted in calling savage Dryops to his Theiodamantine weapons or as cruel Cacus died, crushed, in his cave, given away by the bellowing of oxen inside or Lichas who brought Nessus gift steeped in venom, and stained the Euboean waters with his blood.",The tongue is the instrument of the greatest good and the greatest evil that is in man.,0.843881
455,"Yet every book s not guilty because of it nothing s useful, that can t also wound.",The tongue is the instrument of the greatest good and the greatest evil that is in man.,0.680623
1466,"So may the gods, of whom he himself is the most just, cause kindly earth to create nothing greater than Caesar, and as it has been under his rule, may the earth stay under a Caesar, passed on through the hands of his race.",The tongue is the instrument of the greatest good and the greatest evil that is in man.,0.879356
1315,"Endure, and be true you ve suffered much worse, when the Prince s anger tore me away from you.",The tongue is the instrument of the greatest good and the greatest evil that is in man.,0.652125
946,"His Guilt O you, who are noble in ancestral titles, who outshine your tribe in nobility of character, whose mind mirrors your father s brilliance, yet does not lack a brilliance of its own, whose wit is eloquent in your father s tongue, bettered by no other in the Roman forum little though I wish to do so, I address you with descriptions not names, forgive my praise of you.",The tongue is the instrument of the greatest good and the greatest evil that is in man.,0.833709
1321,"Caesar s power proved lenient to me, as to others, whose number is immeasurable.",The tongue is the instrument of the greatest good and the greatest evil that is in man.,0.764585
940,"129 Who would know of Hector, if Troy had been happy? The road to virtue s paved with public ills.",The tongue is the instrument of the greatest good and the greatest evil that is in man.,0.774261


In [8]:
#########################################################################
# POC: To see if generate a response based on a quote
# Situation: There is a general tweet
# Task: Generate a list of candidate quotes
# Action: Create a model that assigns a score to each quote
# Result: The top 1 quote is returned
##################################################################################################################################################


/var/folders/qq/gdppvj912kv3ds7_xnf0q_fc0000gn/T/ipykernel_36937/2330900647.py:14: UserWarning: [W007] The model you're using has no word vectors loaded, so the result of the Doc.similarity method will be based on the tagger, parser and NER, which may not give useful similarity judgements. This may happen if you're using one of the small models, e.g. `en_core_web_sm`, which don't ship with word vectors and only use context-sensitive tensors. You can always add your own word vectors, or use one of the larger models instead if available.
  similarity_score = doc_generated.similarity(doc_actual)


['Let the man who does not wish to be idle fall in love!',
 'To love and be loved is to feel the sun from both sides.',
 "Love yields to business. If you seek a way out of love, be busy; you'll be safe then.",
 'Happy is the man who has broken the chains which hurt the mind, and has given up worrying once and for all.',
 'Nothing is stronger than habit.',
 'Dripping water hollows out stone, not through force but through persistence.',
 'It is no use to blame the looking glass if your face is awry.',
 'Resist beginnings; the remedy comes too late when the disease has gained strength by long delays.',
 'The mind, conscious of rectitude, laughed to scorn the falsehood of report.',
 'The cause is hidden. The effect is visible to all.']

In [ ]:
df = pd.read_json("./../app/ovid_quotes.json")

DATA_DIR = "./../data"
files = [
    "RemediaAmoris.txt",
    "Heroides.txt",
    "Amours.txt",
    "Fasti.txt",
    "MetamorphosesI_VII.txt",
    "MetamorphosesVIII_XV.txt",
    "MetamorphosesofPublius.txt",
    "LoversAssistant.txt",
    "LastPoems.txt",
]
docs = [f"{DATA_DIR}/{file}" for file in files]
documents = SimpleDirectoryReader(input_files=docs).load_data()

text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=25)
texts = text_splitter.create_documents(docs)
directory = "index_store"
vector_index = FAISS.from_documents(texts, OpenAIEmbeddings())
vector_index.save_local(directory)

vector_index = FAISS.load_local("index_store", OpenAIEmbeddings())
retriever = vector_index.as_retriever(search_type="similarity", search_kwargs={"k": 10})

In [ ]:
df = pd.read_json("./../app/ovid_quotes.json")
df.head(1)

In [ ]:
qa_interface = RetrievalQA.from_chain_type(
    llm=ChatOpenAI(),
    chain_type="stuff",
    retriever=retriever,
    return_source_documents=True,
)

response = qa_interface(
    f"""
I am a big fan of ovid. 
Please recommend 10 memorable quotes to me along with the source document they were taken from.

Do NOT include quotes I already have:
{df["Quote"].tolist()}
"""
)

print(response["result"])

#### Authorship authentication

**[Who Wrote it and Why?
Prompting Large-Language Models for Authorship Verification](https://arxiv.org/pdf/2310.08123.pdf)**

In [ ]:
# LLM Authorship Attribution
from typing import List

from langchain.chat_models import ChatOpenAI
from langchain.prompts import ChatPromptTemplate
from langchain.schema import BaseOutputParser


class CommaSeparatedListOutputParser(BaseOutputParser[List[str]]):
    """Parse the output of an LLM call to a comma-separated list."""

    def parse(self, text: str) -> List[str]:
        """Parse the output of an LLM call."""
        return text.strip().split(", ")


template = f"""Task: On a scale of 0 to 1, with 0 indicating low confidence
and 1 indicating high confidence, please provide a general
assessment of the likelihood that given text 
written by the same author as the provided reference. Your answer should reflect a
moderate level of strictness in scoring. Here are some
relevant variables to this problem.
1. punctuation style(e.g. hyphen, brackets, colon, comma,
parenthesis, quotation mark)
2. special characters style, capitalization style(e.g.
Continuous capitalization, capitalizing certain words)
3. acronyms and abbreviations(e.g. Usage of acronyms
such as OMG, Abbreviations without punctuation marks
such as Mr Rochester vs. Mr. Rochester,Unusual
abbreviations such as def vs. definitely)
4. writing style
5. expressions and Idioms
6. tone and mood
7. sentence structure
8. any other relevant aspect
First step: Understand the problem, extracting relevant
variables and devise a plan to solve the problem. Then,
carry out the plan and solve the problem step by step.
9. One (or both) of the texts is written by the famous Latin author "Ovid"
Finally, show the confidence score.

The following are all quotes by Ovid for reference:
'Love is a thing full of anxious fears.',
 'Now are fields of corn where Troy once stood.',
 "We're slow to believe what wounds us.",
 "The end proves the acts (were done), or the result is a test of the actions; Ovid's line 85 full translation: “The event proves well the wisdom of her [Phyllis'] course.”",
 "Let him who loves, where love success may find, Spread all his sails before the prosp'rous wind.",
 'Resist beginnings; the remedy comes too late when the disease has gained strength by long delays.',
 "Love yields to business. If you seek a way out of love, be busy; you'll be safe then.",
 'The gods behold all righteous actions.',
 'There is a god within us.',
 'The mind, conscious of rectitude, laughed to scorn the falsehood of report.',
 'Every lover is a soldier.',
 'Let the man who does not wish to be idle fall in love!',
 'Far away be that fate!',
 'They bear punishment with equanimity who have earned it.',
 "We take no pleasure in permitted joys. But what's forbidden is more keenly sought.",
 'Who is allowed to sin, sins less.' 
 
 """
human_template = "{text}"

chat_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", template),
        ("human", human_template),
    ]
)
chain = chat_prompt | ChatOpenAI() | CommaSeparatedListOutputParser()

In [ ]:
# Extract Quotes from LLM Authorshipp Attribution
import pandas as pd
import spacy
import re

nlp = spacy.load("en_core_web_sm")
input_string = response["result"]


def get_quotes(input_string):
    """Extract quotes from a string that has been created by an LLM string prompt."""
    # Extract the lines that start with a number
    lines_with_stripped_numbers = [
        re.sub(r"^\d+\.\s*", "", line.strip())
        for line in input_string.splitlines()
        if re.match(r"^\d+\.", line)
    ]

    # Print the extracted lines
    quotes = []
    pattern2 = r'"([^"]+)"\s*-\s*(.+)'

    for line in lines_with_stripped_numbers:
        match = re.search(pattern2, line)
        q = match.group(1)
        w = match.group(2)
        quotes.append((q, w))
    return quotes


quotes = get_quotes(response["result"])
quotes

### Score New Quotes

In [ ]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts import ChatPromptTemplate


def score_quotes(quotes=None):
    """Score a list of quotes that have been generated by an LLM string prompt.

    quotes example:
    [('Dripping water hollows out stone, not through force but through persistence.',
    'Metamorphoses'),

    """
    if quotes is None:
        quotes = []

    # Define the regex pattern
    pattern = r"(\d+\.\d+)"

    # Get the quote string (not the work)
    quotes_text = [q[0] for q in quotes]
    quotes_work = [q[1] for q in quotes]

    # Extract the scores
    scores = []
    score_reasons = []
    for q in quotes_text:
        r = chain.invoke({"text": q})
        r = " ".join(r)
        score_reasons.append(r)
        match = re.search(pattern, r)
        score = match[0] if match else -1
        scores.append(score)
    return list(zip(*(scores, score_reasons, quotes_text, quotes_work)))


# Example usage
scored_quotes = score_quotes(quotes)
scored_quotes_df = pd.DataFrame(
    scored_quotes, columns=["score", "reason", "quote", "work"]
)
pd.set_option('display.max_colwidth', None)
scored_quotes_df  # ["score"]

In [ ]:
scored_quotes_df["quote"].tolist()

### Context-Based Quotation Recommendation

Resource:
* https://arxiv.org/pdf/2005.08319.pdf

### Assessment of Retrieval Accuracy

The below uses current pipeline for 2 reasons:
1) To assess the hulicination affect against current pipeline
2) To assess scoring variability

In [ ]:
import re

non_ovid_generated_quotes = [
    "One man's meat is another man's poison.",
    "Fortune favors the bold.",
    "Wherever there is a human being, there is an opportunity for a kindness.",
    "Love is a kind of warfare.",
    "One man's meat is another man's poison.",
    "To be loved, be lovable.",
]


# Define the regex pattern
pattern = r"(\d+\.\d+)"


scores = []
strings = []
for q in non_ovid_generated_quotes:
    r = chain.invoke({"text": q})
    r = " ".join(r)
    strings.append(r)
    match = re.search(pattern, r)
    score = match[0] if match else -1
    scores.append(score)
    print(r)

pd.DataFrame(
    data={
        "questionable_quote": non_ovid_generated_quotes,
        "authorship_match_score": scores,
        "is_original": [0, 0, 0, 1, 0, 1],
    }
)

In [ ]:
scored_quotes_df[scored_quotes_df["score"].astype(float) < 0.5]["quote"].tolist()

In [ ]:
from itertools import product

# Compare to quotes
generated_quotes = scored_quotes_df[scored_quotes_df["score"].astype(float) > 0.5][
    "quote"
].tolist()

# Read in existing quotes
df = pd.read_json("./../app/ovid_quotes.json")

# Load the spaCy English model
nlp = spacy.load("en_core_web_sm")

# Process each quote and calculate similarity scores
similarity_data = []

for generated_quote, actual_quote in product(generated_quotes, df["Quote"]):
    # Process the quotes with spaCy
    doc_generated = nlp(generated_quote)
    doc_actual = nlp(actual_quote)

    # Calculate similarity score
    similarity_score = doc_generated.similarity(doc_actual)

    # Append data to the list
    similarity_data.append(
        {
            "generated_quotes": generated_quote,
            "actual_quotes": actual_quote,
            "similarity_score": similarity_score,
        }
    )

# Create a DataFrame from the similarity data
similarity_df = pd.DataFrame(similarity_data).sort_values(
    "similarity_score", ascending=False
)
display(similarity_df.head(2))

# Seems like there might be a magic number similarity score < 0.65
print("unique quutes")

similiar_quotes = similarity_df[similarity_df["similarity_score"] > 0.65][
    "generated_quotes"
].unique()

# Filter out the similar quotes
new_quotes = similarity_df[~similarity_df["generated_quotes"].isin(similiar_quotes)][
    "generated_quotes"
].unique()

# Get the works for the new quotes
new_quote_works = []
for q in new_quotes:
    for quote, work in quotes:
        if q == quote:
            new_quote_works.append(work)
df_new_quotes = pd.DataFrame(
    list(zip(*(new_quotes, new_quote_works))), columns=["Quote", "Work"]
)
df_new_quotes["Quote in Latin"] = None
df_new_quotes

In [ ]:
similiar_quotes

In [ ]:
print("highest score comparison:")
idx = 54
print("GENERATED:", similarity_df.iloc[54]["generated_quotes"])
print("ACTUAL:", similarity_df.iloc[54]["actual_quotes"])

### Storage of Additional Quotes

In [16]:
# import pandas as pd

# # Read in the existing dataframe
# df = pd.read_json("./../app/ovid_quotes.json")

# # Manually append a new row to the dataframe
# new_row = {
#     "Quote": "Greet them (others) by their names it costs you nothing.",
#     "Work": "Ars Amatoria",
#     "Quote in Latin": "",
# }
# df = pd.concat([df, pd.DataFrame([new_row])], ignore_index=True)

# # Print the updated dataframe
# display(df.head(2))
# df.to_json("./../app/ovid_quotes.json")
# print("New dataframe shape:", df.shape)

,Quote,Quote in Latin,Work
0,Love is a thing full of anxious fears.,Res est solliciti plena timoris amor.,"Heroides (The Heroines), I, 12"
1,Now are fields of corn where Troy once stood.,Iam seges est ubi Troia fuit.,"Heroides (The Heroines), I, 53"


New dataframe shape: (51, 3)


In [ ]:
# After quotes works have passed the assessment bars above
df = pd.concat([df, df_new_quotes]).reset_index(drop=True)
df.to_json("./../ovid_quotes.json")

## Getting more data

In [95]:
%%bash
ls ../data
curl https://web.seducoahuila.gob.mx/biblioweb/upload/THE%20ART%20OF%20LOVE.pdf > ../data/ArsAmatoria.pdf

AmoresV2.pdf
AmoresV2.txt
Amours.txt
Fasti.txt
FastiV2.pdf
FastiV2.txt
Heroides.pdf
Heroides.txt
HeroidesV2.txt
LastPoems.txt
LostPoemsV2.txt
LoversAssistant.txt
MetamorphosesI_VII.txt
MetamorphosesV2.pdf
MetamorphosesV2.txt
MetamorphosesVIII_XV.txt
MetamorphosesofPublius.txt
RemediaAmoris.txt
lostpoems.pdf
notes.jsonl
notes_validation.jsonl


  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  302k  100  302k    0     0   380k      0 --:--:-- --:--:-- --:--:--  381k


In [96]:
import fitz  # PyMuPDF


def extract_text_from_pdf(pdf_path):
    try:
        # Open the PDF file
        pdf_document = fitz.open(pdf_path)

        # Initialize an empty string to store the extracted text
        extracted_text = ""

        # Iterate through pages and extract text
        for page_number in range(pdf_document.page_count):
            page = pdf_document[page_number]
            extracted_text += page.get_text()

        return extracted_text

    except Exception as e:
        print(f"Error: {e}")
        return None


# Example usage:
pdf_file_path = "../data/ArsAmatoria.pdf"  # Replace with the path to your PDF file
extracted_text = extract_text_from_pdf(pdf_file_path)

In [97]:
print(len(extracted_text.split()))
extracted_text.split()[-1000:-990]

21875


['eat.',
 'As',
 'her',
 'breath',
 'returned,',
 'she',
 'tore',
 'the',
 'thin',
 'clothing']

In [98]:
file_path = "../data/ArsAmatoria.txt"

with open(file_path, "w") as file:
    file.write(extracted_text)

print("Text file created successfully.")

Text file created successfully.


### LLM Quote Retrieval

In [81]:
from langchain.document_loaders import PyPDFLoader
from langchain.vectorstores import FAISS
from langchain.chat_models import ChatOpenAI
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.chains import RetrievalQA, ConversationalRetrievalChain
import os
import openai

# import utils
import configparser
from pprint import pprint
from llama_index import SimpleDirectoryReader


f1 = "./../data/LostPoemsV2.txt"
f2 = "./../data/FastiV2.txt"

docs = [f1, f2]
documents = SimpleDirectoryReader(input_files=docs).load_data()

text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=25)
texts = text_splitter.create_documents(docs)
directory = "index_store"
vector_index = FAISS.from_documents(texts, OpenAIEmbeddings())
vector_index.save_local(directory)

vector_index = FAISS.load_local("index_store", OpenAIEmbeddings())
retriever = vector_index.as_retriever(search_type="similarity", search_kwargs={"k": 6})

In [82]:
qa_interface = RetrievalQA.from_chain_type(
    llm=ChatOpenAI(),
    chain_type="stuff",
    retriever=retriever,
    return_source_documents=True,
)

response = qa_interface(
    """
I am a big fan of ovid. 
Please recommend memorable quotes to me.
"""
)

print(response["result"])

I apologize, but as an AI language model, I do not have the ability to browse specific files or access specific quotes from works of literature. I recommend reading Ovid's works, such as "Lost Poems" and "Fasti," to discover and appreciate the memorable quotes within them.
